<a href="https://colab.research.google.com/github/kavishajain5/Ineuron-Internship-/blob/main/sub_ineuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile kaggle.json
{"username":"rushikeshdarge","key":"*********************************"}

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c learn-ai-bbc
! unzip /content/learn-ai-bbc.zip

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, SpatialDropout1D
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Downloading necessary NLTK resources
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

nltk.download('wordnet')
nltk.download('omw-1.4')


In [ ]:
df = pd.read_csv('/content/BBC News Train.csv')
df.head()

In [ ]:
df.Category.value_counts()

In [ ]:
df.isnull().sum()

EDA

In [ ]:
categories = ['business', 'politics', 'entertainment', 'sport', 'tech']
df['Category'] = df['Category'].replace(categories, [0, 1, 2, 3, 4])

In [ ]:
wc_stopwords = set(STOPWORDS)

business_data = df[df['Category'] == 0].Text
wordcloud = WordCloud(stopwords = wc_stopwords, max_words = 200, background_color = 'white', width = 1200, height = 800).generate(" ".join(business_data))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.title('WordCloud for Business News Articles\n')
plt.axis('off')
plt.show()


In [ ]:
education_data = df[df['Category'] == 1].Text
wordcloud = WordCloud(stopwords = wc_stopwords, max_words = 200, background_color = 'white', width = 1200, height = 800).generate(" ".join(education_data))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.title('WordCloud for Polical News Articles\n')
plt.axis('off')
plt.show()


In [ ]:
entertainment_data = df[df['Category'] == 2].Text
wordcloud = WordCloud(stopwords = wc_stopwords, max_words = 200, background_color = 'white', width = 1200, height = 800).generate(" ".join(entertainment_data))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.title('WordCloud for Entertainment News Articles\n')
plt.axis('off')
plt.show()


In [ ]:
sports_data = df[df['Category'] == 3].Text
wordcloud = WordCloud(stopwords = wc_stopwords, max_words = 200, background_color = 'white', width = 1200, height = 800).generate(" ".join(sports_data))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.title('WordCloud for Sports News Articles\n')
plt.axis('off')
plt.show()


In [ ]:
technology_data = df[df['Category'] == 4].Text
wordcloud = WordCloud(stopwords = wc_stopwords, max_words = 200, background_color = 'white', width = 1200, height = 800).generate(" ".join(technology_data))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.title('WordCloud for Technology News Articles\n')
plt.axis('off')
plt.show()


Text preprocessing

In [ ]:
def chars(text):
    return re.sub('[^A-Za-z0-9 ]+', '', text)

def decontractions(phrase):
    """Decontracted takes text and converts contractions into their natural form."""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    return phrase

def stopwords(text):
    text = text.split(' ')
    output = [i for i in text if i not in stopwords]
    return ' '.join(output)
def stemming(text):
    text = text.split(' ')
    stem_text = [porter_stemmer.stem(word) for word in text]
    return ' '.join(stem_text)
def lemmatizer(text):
    text = text.split(' ')
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return ' '.join(lemm_text)

def preprocessing(text_df):
    text_df = text_df.apply(lambda x : chars(x))
    text_df = text_df.apply(lambda x : x.lower())
    text_df = text_df.apply(lambda x : decontractions(x))
    text_df = text_df.apply(lambda x: remove_stopwords(x))
    text_df = text_df.apply(lambda x: stemming(x))
    text_df = text_df.apply(lambda x: lemmatizer(x))
    return text_df
